In [1]:
from IPython.display import display, HTML

display(
    HTML(
        """<style>
* {font-family:D2Coding;}
div.container{width:87% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input { font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:3px;}
table.dataframe{font-size:12px;}
</style>
"""
    )
)

[ RAG 구현 절차 ]

1. 문서의 내용을 읽는다(document_loader를 이용)

- (1) https://python.langchain.com/v0.2/docs/integrations/document_loaders/
- (2) https://python.langchain.com/v0.2/docs/integrations/document_loaders/microsoft_word/
- %pip install --upgrade --quiet docx2txt

2. 문서를 쪼갠다(한번에 이해하고 처리할 수 있는 입력+출력 토큰수가 제한)

- (1) https://python.langchain.com/v0.2/docs/how_to/recursive_text_splitter/#splitting-text-from-languages-without-word-boundaries
- %pip install -qU langchain-text-splitters

3. 쪼갠 문서를 임베딩하여 vector database에 넣음

- (1) OpenAIEmbeddings나 UpstageEmbeddings이용해서 임베딩
- (2) https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/
- %pip install –q langchain-chroma

4. 질문을 이용해 유사도 검색
5. 유사도 검색한 문서를 LLM에 질문으로 전달하여 답변 얻음(제공되는 Prompt활용)

- (1) https://python.langchain.com/v0.2/docs/tutorials/rag/
- %pip install –q langchain langchainhub

- https://smith.langchain.com/ 에서 key 생성 .env key 추가


In [ ]:
# 문서 읽어오기
# %pip install --upgrade --quiet docx2txt

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# 텍스트를 정크로 나누는 기능만 있는 경량 모듈
# %pip install -qU langchain-text-splitters

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# 벡터 데이터베이스 (로컬 데이터베이스)
# %pip install -q langchain-chroma

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# 제공 되는 prompt 사용
# %pip install -q langchain langchainhub

Note: you may need to restart the kernel to use updated packages.


<!-- # 1. 문서 읽기 (X) -->


In [9]:
from langchain_community.document_loaders import Docx2txtLoader

loader = Docx2txtLoader("./tax_docs/소득세법(법률)(제20615호)(20250701).docx")
document = loader.load()
# documents


In [11]:
len(document)

1

In [12]:
document[0].page_content[:200]

'소득세법\n\n소득세법\n\n[시행 2025. 7. 1.] [법률 제20615호, 2024. 12. 31., 일부개정]\n\n기획재정부(재산세제과(양도소득세)) 044-215-4312\n\n기획재정부(소득세제과(근로소득)) 044-215-4216\n\n기획재정부(금융세제과(이자소득, 배당소득)) 044-215-4233\n\n기획재정부(소득세제과(사업소득, 기타소득)) 044-2'

# 2. 문서를 쪼개면서 읽기 (O)


In [13]:
import time

start = time.time()

from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import (
    RecursiveCharacterTextSplitter,
)  # 문서를 읽어오면서 겹치게 발라줌

loader = Docx2txtLoader("./tax_docs/소득세법(법률)(제20615호)(20250701).docx")

# 문자단위로 쪼갬
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, chunk_overlap=200  # 문서를 쪼갤떄 1500글자씩 쪼개
)
# 1번째 chunk 1 ~ 1450글자
# 2번쨰 chunk 1250 ~ 1750글자

document = loader.load_and_split(text_splitter=text_splitter)

runtime = time.time() - start
print("문서 쪼개면서 읽는 시간", runtime)

문서 쪼개면서 읽는 시간 1.6555850505828857


In [14]:
# chunk 갯수
len(document)

183

In [15]:
len(document[0].page_content)

1464

In [16]:
# chunk의 글자수들
# [len(doc.page_content) for doc in document]
print(max(len(doc.page_content) for doc in document))
print(min(len(doc.page_content) for doc in document))

1497
1055


# 3. 쪼갠문서를 임베딩 -> 벡터 데이터베이스 저장

- Embedding Model : openAI API text-embedding-3-large (기본 : text-embedding-ada-002)
- 벡터 데이터베이스 : chroma


In [17]:
# 사용법 : https://python.langchain.com/v0.2/docs/how_to/embed_text/
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()
#
embedding = OpenAIEmbeddings(
    model="text-embedding-3-large",
)

In [18]:
embeddings = embedding.embed_documents(
    ["소득세법 어쩌구 저쩌구", document[0].page_content]
)

In [19]:
len(embeddings), len(embeddings[0]), len(embeddings[1])

(2, 3072, 3072)

In [20]:
len(embeddings)

2

In [21]:
%%time
from langchain_chroma import Chroma

# 데이터를 처음 저잘할떄
# database = Chroma.from_documents(
#     documents=document,
#     embedding=embedding,
#     collection_name='tax_collection',# 생략시 이름 랜덤
#     persist_directory='./chroma',# 생략시 로컬데이터베이스에 저장안됨, 프로그램 종료시 db날라감
# )
# 계속 실행하면 하면 안됨. 꼭 한번만. 그래서 주석처리


CPU times: user 155 ms, sys: 22 ms, total: 177 ms
Wall time: 206 ms


In [22]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
%time
load_dotenv()
embedding = OpenAIEmbeddings(
    model="text-embedding-3-large",
)

# 이미 저장된 vector DB를 사용할떄
database = Chroma(
    embedding_function=embedding,
    collection_name='tax_collection',
    persist_directory='./chroma'
)

CPU times: user 2 μs, sys: 1e+03 ns, total: 3 μs
Wall time: 5.72 μs


# 4. vector DB 에 질문과 유사도 검색 (답변 생성을 위한 retrieval)


In [23]:
query = "연봉 5000만원인 직장인의 소득세는 얼마인가요?"

retrieved_docs = database.similarity_search(query, k=3)  # 기본 k값이 4

# 5. 유사도 검색으로 가져온 문서를 질문과 같이 LLM 전달하여 답변 생성


In [24]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-nano")

In [25]:
prompt = f"""[identity]
- 당신은 최고의 한국 소득세 전문가입니다.
- [context]를 참고해서 사용자의 질문에 답변해 주세요.
[context]는 다음과 같습니다.
{retrieved_docs}
Question: {query}
"""

In [26]:
ai_message = llm.invoke(prompt)

In [27]:
print(ai_message.content)

연봉이 5,000만 원인 직장인의 경우, 소득세는 과세표준과 세율에 따라 계산됩니다. 기본적으로 근로소득세 계산 방법은 아래와 같습니다.

1. 근로소득 공제액 산정  
연봉에 따라 공제액이 차감됩니다. 2023년 기준으로 연봉 5,000만 원인 경우 근로소득 공제액은 약 1,230만 원입니다.  
(공제액은 매년 조정되기 때문에 최신 공제액은 국세청 자료를 참고하는 것이 좋습니다.)

2. 과세표준 계산  
과세표준 = 연봉 - 근로소득 공제액 = 50,000,000원 - 12,300,000원 = 37,700,000원

3. 세율 적용  
2023년 한국의 근로소득세 세율 구간에 따라 계산  
- 1,200만 원 이하: 6%  
- 1,200만 원 초과 ~ 4,600만 원 이하: 15% (초과분에 대해 적용)

계산순서:  
- 1,200만 원까지: 1,200만 원 x 6% = 72만 원  
- 1,200만 원 초과 ~ 3,770만 원(과세표준): (3,770만 원 - 1,200만 원) = 2,570만 원  
- 15% 세율 적용액: 2,570만 원 x 15% = 385.5만 원

이때, 세액 계산 공식은 누진 공제액을 고려하는데, 이를 반영하면 최종 소득세는 대략 50만 ~ 60만 원대가 될 수 있습니다.

**대략적인 계산 예시:**  
소득세 ≈ (72만 원 + 385.5만 원) - 세액공제(근로소득세액공제 등)  
세액공제는 개인 상황에 따라 다르지만, 대체로 10~20% 정도 공제받을 수 있으니 최종 세금은 약 46만 원 내외로 예상됩니다.

**참고:**  
- 지방소득세(소득세의 10%)가 별도로 부과됩니다.  
- 정확한 금액은 개인 소득공제, 연금보험료, 의료비, 기타 공제 요소에 따라 차이 날 수 있습니다.

**요약:**  
연봉 5,000만 원인 직장인의 예상 소득세는 대략 46만 원 내외입니다. 정확한 계산을 원하시면, 연말 정산 자료와 공제 내역을 반영해 세무사 또는 국세청 홈택스의 계산기를 이용하시길 권장드립니다.


# 5. Agumentation 을 위한 제공하는 Prompt 활용하여, langchain으로 답변 생성

- 사용법 : https://python.langchain.com/v0.2/docs/tutorials/rag/


In [28]:
query = "연봉 5000만원인 직장인의 소득세는 얼마인가요?"

from langchain import hub

prompt = hub.pull("rlm/rag-prompt")
prompt

/Users/teamkim/anaconda3/envs/llm/lib/python3.10/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

## RetrievalQA를 통해 LLM전달 (create_retrieval_chain이 대체)

```
 query -> retriever 전달(벡터 검색 수행)
 -> retriever문서 -> prompt의 {context}에 삽입
 -> query -> prompt의 (question)에 삽입
```


In [29]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=database.as_retriever(search_kwargs={"k": 5}),
    chain_type_kwargs={"prompt": prompt},
)

In [30]:
ai_message = qa_chain.invoke({"query": query})

In [31]:
ai_message

{'query': '연봉 5000만원인 직장인의 소득세는 얼마인가요?',
 'result': '소득세는 소득 수준, 공제액 등에 따라 달라지기 때문에 정확한 금액을 계산하려면 자세한 세액 계산이 필요합니다. 일반적으로 연봉 5000만원인 경우, 근로소득세는 차감 공제 후 과세표준에 따라 결정됩니다. 따라서 정확한 금액은 세법과 공제 조건을 고려한 계산이 필요하며, 구체적인 세액을 알려면 세무 전문가 또는 세금 계산기를 이용하는 것이 좋습니다.'}